In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
import glob
from skimage.transform import resize
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
test_dicoms = glob.glob('*.dcm')


In [3]:
a = pydicom.dcmread(test_dicoms[0])


In [4]:
for i in test_dicoms:
    ds = pydicom.dcmread(i)
    print(ds.StudyDescription, ds.Modality, ds.PatientPosition)

No Finding CT PA
No Finding DX PA
No Finding DX XX
Cardiomegaly DX AP
Effusion DX AP
No Finding DX PA


In [5]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    idg = ImageDataGenerator()
    img = resize(img, IMG_SIZE)
    proc_img = idg.flow(
            img,
            y=None,
            batch_size=32,
            shuffle=True,
            sample_weight=None,
            seed=None,
            save_to_dir=None,
            save_prefix="",
            save_format="png",
            subset=None,)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    file = open(model_path, 'r')
    my_model = file.read()
    file.close()
    
    model = model_from_json(my_model)
    #model = weight_path
    model.load_weights(weight_path)

    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo 
    predn_prob = model.predict(img, verbose = True)
    prediction = ['There is pneumonia' if i[0] > thresh else 'No pneumonia in this image' for i in predn_prob]
    
    return predn_prob, prediction 

In [6]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json' #path to saved model
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 2646.035253 # loads the mean image value they used during training preprocessing
img_std = 341.2437# loads the std dev image value they used during training preprocessing
 
my_model = load_model(model_path, weight_path)#loads model
thresh = 0.6 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
1/1 [==============================] - 2s 2s/step
(array([[0.45933303]], dtype=float32), ['No pneumonia in this image'])
Load file test2.dcm ...
1/1 [==============================] - 0s 37ms/step
(array([[0.45155138]], dtype=float32), ['No pneumonia in this image'])
Load file test3.dcm ...
1/1 [==============================] - 0s 36ms/step
(array([[0.45902377]], dtype=float32), ['No pneumonia in this image'])
Load file test4.dcm ...
1/1 [==============================] - 0s 36ms/step
(array([[0.45933303]], dtype=float32), ['No pneumonia in this image'])
Load file test5.dcm ...
1/1 [==============================] - 0s 36ms/step
(array([[0.45933303]], dtype=float32), ['No pneumonia in this image'])
Load file test6.dcm ...
1/1 [==============================] - 0s 35ms/step
(array([[0.45933303]], dtype=float32), ['No pneumonia in this image'])


In [ ]:
## Seems like the model is doing very bad :(